In [ ]:
!pip -q install huggingface chromadb transformers langchain

In [2]:
!pip -q install configparser

In [27]:
!pip -q install openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.2 MB/s eta 0:00:00


In [4]:
import os
import configparser

data = configparser.ConfigParser()
data.read_file(open('/content/apidata.config'))
os.environ['HUGGINGFACEHUB_API_TOKEN'] = data["OPENAI"]["HFREAD"]
os.environ['SERPAPI_API_KEY'] = data["OPENAI"]["SERPAPI_KEY"]
os.environ['OPENAI_API_KEY'] = data["OPENAI"]["KEY"]

In [5]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader

In [9]:
spaceLoad = TextLoader('/content/space_shortened.csv')

In [7]:
from langchain.text_splitter import CharacterTextSplitter

In [10]:
documents = spaceLoad.load()

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
recurSplitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                               chunk_overlap=20,
                                               length_function=len)

In [12]:
with open('/content/space_shortened.csv') as lin:
  txt_lin = lin.read()

recurse_documents = recurSplitter.create_documents([txt_lin])

In [13]:
recurse_documents[0]

Document(page_content='PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VR', metadata={})

# Now work on embedding

In [17]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
# Supplying a persist_directory will store the embeddings on disk

charSplit_directory = 'charSplit_db'

plain_chroma = Chroma.from_documents(documents=recurse_documents,
                                     embeddings=OpenAIEmbeddings(),
                                     persist_directory=charSplit_directory)

In [20]:
query = "EARTH"
plain_chroma.similarity_search(query)

[Document(page_content='0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True', metadata={}),
 Document(page_content='0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True', metadata={}),
 Document(page_content='0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True', metadata={}),
 Document(page_content='0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True', metadata={})]

In [21]:
plain_chroma.similarity_search_with_score(query)

[(Document(page_content='0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True', metadata={}),
  1.2871034145355225),
 (Document(page_content='0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True', metadata={}),
  1.3029760122299194),
 (Document(page_content='0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True', metadata={}),
  1.3070783615112305),
 (Document(page_content='0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,,Candra Jacostaffey,True', metadata={}),
  1.3312859535217285)]

#Startin to use LLMs and Chains

In [23]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import AnalyzeDocumentChain
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

In [24]:
from langchain.chains.summarize import load_summarize_chain

factLLM = OpenAI(temperature=0.01)

summary_chain = load_summarize_chain(llm=factLLM, 
                                     chain_type="map_reduce")

In [28]:
summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)
summarize_document_chain.run(txt_lin)

' This data set contains information about 12 passengers travelling to the TRAPPIST-1e planet, including their home planet, age, VIP status, and usage of various services. It also includes their name and whether they were transported or not.'

In [31]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=factLLM, 
                                 chain_type="stuff", 
                                 retriever=plain_chroma.as_retriever())

In [32]:
qa.Config()

In [34]:
qa.run("How many are going to Trappist?")

' Four people are going to Trappist.'

In [36]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

In [38]:
vectorstore_info = VectorStoreInfo(
    name="space_farers",
    description="the most recent space farers data",
    vectorstore=plain_chroma
)

In [39]:
from langchain.chains import RetrievalQAWithSourcesChain

In [40]:
toolkit = VectorStoreToolkit(llm=factLLM,
                             vectorstore_info=vectorstore_info)

agent_executor = create_vectorstore_agent(
    llm=factLLM,
    toolkit=toolkit,
    verbose=True
)

In [42]:
agent_executor.run("What are different space destinations?")



> Entering new AgentExecutor chain...
 I need to find out what space destinations are available.
Action: space_farers
Action Input: What are different space destinations?
Observation:  The two destinations mentioned in the context are Earth and TRAPPIST-1e.
Thought: I now know the final answer.
Final Answer: The two destinations mentioned in the context are Earth and TRAPPIST-1e.

> Finished chain.


'The two destinations mentioned in the context are Earth and TRAPPIST-1e.'